# Collection of article data from L/N database service

1. collect
2. process html content on the fly
3. dump in mysql database

In [2]:
# import basics
import os
import re
import csv
import json
import time
import requests
import pandas as pd

from time import sleep
from datetime import datetime 
from dotenv import load_dotenv
from requests.auth import HTTPBasicAuth

# text processing dependencies
from bs4 import BeautifulSoup
#from boilerpy3 import extractors

# mysql dependencies
import mysql.connector

# collection dependencies
#from lexis import *


In [32]:
# initialize modules and variables

load_dotenv(override=True)
#extractor = extractors.ArticleExtractor()

# L/N credentials
CLIENT = os.getenv("CLIENTID")
SECRET = os.getenv("SECRET")

# mysql credentials
PASSWORD = os.getenv("PASSWORD")
USER = os.getenv("USER")

# define query
QUERY = "privacy"

# request token
token = get_token(CLIENT, SECRET)
request_headers = build_header(token)

Token retrieved successfully.


In [3]:
current_stats = pd.read_json("privacy_collection_stats.json")

In [104]:
#current_stats

In [4]:
def collect(n, y, k, s=0, t=50): # newspaper, year
    
    # create filter
    ###fltr = "Source/Name eq '"+ newspapers[n] + "' and " + "year(Date) eq " + str(y) + ""
    fltr = "Source/Name eq '"+ newspapers[n] + "' and " + "year(Date) eq " + str(y) + ""
    while True:
        # create request (top maxes at 50)
        request_url = build_url(content='News', query=QUERY, skip=s, top=t, filter=fltr)

        r = requests.get(request_url, headers=request_headers)
        json_data = r.json() # extract data
        total = get_result_count(json_data) # total articles per given year/newspaper

        if s == 0:
            print("Total\t", n, "-", y, ": ", total, sep="") 

           

        for i in range(len(json_data["value"])):
            aid = n + "_" + str(k)
            articles[y][aid] = process_article(json_data["value"][i], aid)
            k += 1
        
    
        if s > total: #s == total for test
            break
        else:
            print("Processing... ", "Left:\t", total - s)    
            s = (s + t)
            sleep(18)
        
        
    return k # json_data

In [5]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [6]:
def extract_content(raw_article):
        
    cln = cleanhtml(str(BeautifulSoup(raw_article["Document"]["Content"]).findAll("bodytext")[-1]))

    x = first(cln[:30])
    #print(articles[2011][i]["content"][x:20])
    return cln[x:]#cleanhtml(str(BeautifulSoup(raw_article["Document"]["Content"]).findAll("bodytext")[-1]))

def extract_summary(raw_article):
    
    summary = {
        "extract": "",
        "terms": []
    }

    # list
    for ext in raw_article["Extracts"]:
        if ext["Type"] == "Extract":
            summary["extract"] = cleanhtml(str(BeautifulSoup(ext["SummaryText"])))
        else:
            summary["terms"].append(cleanhtml(str(BeautifulSoup(ext["SummaryText"]))))

    return summary 

def extract_metadata(raw_article):
    
    metadata = {
        "date": raw_article["Date"],
        "title": raw_article["Title"],
        "wordCount": raw_article["WordLength"],
        "section": raw_article["Section"]
    }
    
    return metadata

def extract_date(raw_article):
    return datetime.strptime(raw_article["Date"], '%Y-%m-%dT%H:%M:%SZ')

In [7]:
def process_article(a, aid):
    
    article = {
        "newspaper": n,
        "article_id": aid,
        "title": get_clean_content(extract_metadata(a)["title"]),
        "content": get_clean_content(extract_content(a)),
        "summary": extract_summary(a),
        "metadata": extract_metadata(a),
        "date": extract_date(a),
        "stage": "",
        "endeavor": "",
        "tone": "",
        "tone_result": "",
        "is_privacy": ""
    }
    
    return article 

In [8]:
#newspapers["TDP"] = "The Dominion Post (Wellington, New Zealand)"

In [9]:
#articles[2017]["USA_1963"]

In [6]:
# read newspaper info

newspapers = {}

with open('./newspapers.csv', 'r') as f:
        reader = csv.reader(f)
        for r in reader:
            newspapers[r[0]] = r[1]
        

In [8]:
newspapers["AFR"]

'Australian Financial Review'

In [28]:
n = "NYT"
y = 2015
    
print("YEAR", y)
key = collect(n, y, key, s=0, t=50) # s = skip_value
    
#     print("Dropping to db...")
#     for a in articles[y].values():
#         try:
#             insert_article(a, mycursor, mydb)
#         except Exception as e: 
#             print(e)

YEAR 2015
Total	NYT-2015: 475
Processing...  Left:	 475


In [39]:
batches = {}
resids = []

In [ ]:

def build_url(content='News', query='', skip=0, expand='Document', top=50, filter=None):
    """Builds the URL part of the request to Web Services API."""
    if filter != None:
        api_url = ('https://services-api.lexisnexis.com/v1/' + content +
                    '?$expand=' + expand + '&$search=' + query +
                    '&$skip=' + str(skip) + '&$top=' + str(top) +
                    '&$filter=' + filter)
    else:
        api_url = ('https://services-api.lexisnexis.com/v1/' + content +
                    '?$expand=' + expand + '&$search=' + query +
                    '&$skip=' + str(skip) + '&$top=' + str(top))
    return api_url

In [107]:
def geturl(s, t, filter=None):
    return ('https://services-api.lexisnexis.com/v1/News?$orderby=Date&$expand=Document&$search=privacy' + 
'&$skip=' + str(s) + '&$top=' + str(t) + '&$filter=' + filter)

In [ ]:


GET https://services-api.lexisnexis.com/v1/
    News?$expand=PostFilters,
    Source&$search=privacy&$filter=year(Date)+eq+2016




GET https://services-api.lexisnexis.com/v1/
    News?$orderby=Date&$expand=Document&$search=privacy



In [98]:
articles = {}

In [124]:
fltr = "Source/Name eq '"+ "The New York Times" + "' and " + "year(Date) eq " + str(2016) + ""
s = 0
request_url = geturl(s, 50, filter=fltr)
print(request_url)
r = requests.get(request_url, headers=request_headers)
json_data = r.json() # extract data
total = json_data["@odata.count"] # total articles per given year/newspaper
print(total)
articles[s] = json_data["value"]

https://services-api.lexisnexis.com/v1/News?$orderby=Date&$expand=Document&$search=privacy&$skip=0&$top=50&$filter=Source/Name eq 'The New York Times' and year(Date) eq 2016
98


In [110]:
for i in articles[0]:
    print(i["Date"])

2016-01-04T00:00:00Z
2016-01-05T00:00:00Z
2016-01-05T00:00:00Z
2016-01-05T00:00:00Z
2016-01-05T00:00:00Z
2016-01-06T00:00:00Z
2016-01-07T00:00:00Z
2016-01-08T00:00:00Z
2016-01-09T00:00:00Z
2016-01-09T00:00:00Z
2016-01-13T00:00:00Z
2016-01-16T00:00:00Z
2016-01-19T00:00:00Z
2016-01-22T00:00:00Z
2016-01-22T00:00:00Z
2016-01-29T00:00:00Z
2016-01-29T00:00:00Z
2016-01-29T00:00:00Z
2016-02-02T00:00:00Z
2016-02-05T00:00:00Z
2016-02-06T00:00:00Z
2016-02-09T00:00:00Z
2016-02-19T00:00:00Z
2016-02-23T00:00:00Z
2016-02-26T00:00:00Z
2016-02-27T00:00:00Z
2016-03-03T00:00:00Z
2016-03-05T00:00:00Z
2016-03-08T00:00:00Z
2016-03-08T00:00:00Z
2016-03-10T00:00:00Z
2016-03-10T00:00:00Z
2016-03-12T00:00:00Z
2016-03-14T00:00:00Z
2016-03-14T00:00:00Z
2016-03-15T00:00:00Z
2016-03-15T00:00:00Z
2016-03-16T00:00:00Z
2016-03-22T00:00:00Z
2016-03-24T00:00:00Z
2016-03-24T00:00:00Z
2016-03-29T00:00:00Z
2016-03-29T00:00:00Z
2016-04-01T00:00:00Z
2016-04-01T00:00:00Z
2016-04-04T00:00:00Z
2016-04-08T00:00:00Z
2016-04-18T00

In [114]:
for i in articles[50]:
    print(i["Date"])

2016-04-21T00:00:00Z
2016-04-22T00:00:00Z
2016-04-23T00:00:00Z
2016-04-23T00:00:00Z
2016-04-29T00:00:00Z
2016-04-29T00:00:00Z
2016-04-29T00:00:00Z
2016-04-29T00:00:00Z
2016-05-02T00:00:00Z
2016-05-03T00:00:00Z
2016-05-14T00:00:00Z
2016-05-17T00:00:00Z
2016-05-17T00:00:00Z
2016-05-19T00:00:00Z
2016-05-20T00:00:00Z
2016-05-23T00:00:00Z
2016-05-23T00:00:00Z
2016-05-24T00:00:00Z
2016-05-25T00:00:00Z
2016-05-27T00:00:00Z
2016-06-04T00:00:00Z
2016-06-07T00:00:00Z
2016-06-07T00:00:00Z
2016-06-07T00:00:00Z
2016-06-08T00:00:00Z
2016-06-10T00:00:00Z
2016-06-16T00:00:00Z
2016-06-17T00:00:00Z
2016-06-17T00:00:00Z
2016-06-17T00:00:00Z
2016-06-18T00:00:00Z
2016-06-18T00:00:00Z
2016-06-18T00:00:00Z
2016-06-21T00:00:00Z
2016-06-21T00:00:00Z
2016-06-25T00:00:00Z
2016-06-25T00:00:00Z
2016-06-27T00:00:00Z
2016-07-07T00:00:00Z
2016-07-13T00:00:00Z
2016-07-14T00:00:00Z
2016-07-14T00:00:00Z
2016-07-14T00:00:00Z
2016-07-20T00:00:00Z
2016-07-22T00:00:00Z
2016-07-26T00:00:00Z
2016-07-26T00:00:00Z
2016-07-26T00

In [119]:
mega_batch = []

In [121]:
for i in articles[50]:
    mega_batch.append(i)

In [123]:
titles = []
for i in mega_batch:
    if i["Title"] not in titles:
        titles.append(i["Title"])
    else:
        print(i["Title"], i["Date"], i["WordLength"])

Private health funds 'misuse dental data' 2016-07-26T00:00:00Z 517
Private health funds 'misuse dental data' 2016-07-26T00:00:00Z 517
Private health funds 'misuse dental data' 2016-07-26T00:00:00Z 517
Private health funds 'misuse dental data' 2016-07-26T00:00:00Z 517


In [66]:
batches[0][10].keys()

dict_keys(['Date', 'CompanyName', 'Geography', 'ContentType', 'GroupDuplicates', 'Industry', 'Publisher', 'Title', 'Location', 'LNGI', 'WebNewsUrl', 'SearchType', 'Company', 'NegativeNews', 'IsCitationMatch', 'DocumentContent@odata.mediaContentType', 'Language', 'Section', 'Subject', 'Document', 'People', 'PublicationType', 'Jurisdiction', 'Overview', 'Extracts', 'Source', 'DocumentContent@odata.mediaReadLink', 'LEI', 'InternationalLocation', 'Keyword', 'WordLength', 'Byline', 'ResultId'])

In [77]:
batches[0][10]["ResultId"]

'urn:contentItem:5KVT-PWJ1-JD34-V4XK-00000-00'

In [82]:
len(set(resids))

200

In [93]:
titles = []


In [96]:
for i in batches[150]:
    if i["Title"] not in titles:
        titles.append(i["Title"])
    else:
        print(i["Title"], i["Date"], i["WordLength"])
        print(i["ResultId"])

NAB, Westpac join Ripple network 2016-09-16T00:00:00Z 486
urn:contentItem:5KPT-TK01-JD34-V511-00000-00
Property sector warns against more foreign investment rules 2016-09-21T00:00:00Z 432
urn:contentItem:5KSG-DXH1-JD34-V3YD-00000-00
Property sector warns against more foreign investment rules 2016-09-21T00:00:00Z 432
urn:contentItem:5KSH-1P91-JD34-V27D-00000-00
Rich-lister Gandel keeps on shopping;Retail 2016-09-24T00:00:00Z 563
urn:contentItem:5KSG-DXG1-JD34-V1D6-00000-00
Rich-lister Gandel keeps on shopping;Retail 2016-09-24T00:00:00Z 563
urn:contentItem:5KSG-HSX1-JD34-V18P-00000-00
Rich-lister Gandel keeps on shopping;Retail 2016-09-24T00:00:00Z 563
urn:contentItem:5KSG-D7V1-F0J6-J4JV-00000-00
We're all collateral damage in trans-Atlantic regulatory war;Washington observed 2016-09-20T00:00:00Z 797
urn:contentItem:5KRN-1SX1-JD34-V3YK-00000-00


In [97]:
len(set(titles))

136

In [89]:
titles

['Password tiff between CBA and Acorns',
 "Private health funds 'misuse dental data'",
 "Private health funds 'misuse dental data'",
 "Private health funds 'misuse dental data'",
 "Private health funds 'misuse dental data'",
 "Private health funds 'misuse dental data'",
 "Queenslanders push for PM's top spots",
 'Room to party;HOTEL- REVIEW',
 'Selfie drones all set for take-off;High tech',
 'Simply the best BAGLIONI HOTEL REGINA, ROME;ITALY - TRAVEL',
 'Single secure online access the way to go;Digital identity - Hacking',
 'THE APOCALYPSE WE HAD TO HAVE;Culture',
 'Turnbull says little point to Briggs inquiry',
 "WE'RE ON THE CUSP OF AN AI REVOLUTION",
 "Why Jeremy Clarkson won't want a fully autonomous vehicle;Mobility",
 '60,000 caught in NAB email breakdown;Banking',
 'AFL is richer for Greater Western Sydney;The Adventures of Joe Aston',
 'AFTER THE AFFAIR',
 'A LONG ROAD FOR KIDMAN CLAN;Property',
 'Anchorage called to the rescue of Dick Smith;Retail',
 "Apple's tax battles far 

In [29]:
def first(str) :
     
    for i in range(0, len(str)) :
         
        if (str[i].istitle()) :
            return i#str[i]
             
    return 0

In [46]:
# for i in articles[2011].keys():
#     x = first(articles[2011][i]["content"][:20])
#     print(articles[2011][i]["content"][x:20])

In [46]:
years = [2020]

In [47]:
#years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

key = 0

articles = {}
for y in years:
    articles[y] = {}

In [48]:
# for i in articles[2011].keys():
#     print(articles[2011][i]["content"][:20])

In [49]:
not_added = {}

In [112]:
# for i in not_added.keys():
#     not_added[i]["title"] = get_clean_content(not_added[i]["title"])
#     try:
#         insert_article(not_added[i], mycursor, mydb)
#     except Exception as e: 
#         print(i)
#         print(e)

In [14]:
#s = "some\x00string. with\x15 funny characters"
import string
printable = set(string.printable)
#filter(lambda x: x in printable, s)
#'somestring. with funny characters'

In [35]:
def get_clean_content(s):
    return ''.join(filter(lambda x: x in printable, s))

In [50]:
years


[2020]

In [55]:
### 2 modify
# per newspaper

key = 6919
#years = [2018, 2019, 2020]

n = "TS"
#years = [2011, 2012, 2013]

for y in years:
    
    print("YEAR", y)
    key = collect(n, y, key, s=0, t=50) # s = skip_value
    
    print("key: ", key)
        
    print("Dropping to db...")
    for a in articles[y].values():
        try:
            insert_article(a, mycursor, mydb)
        except Exception as e: 
            not_added[a["article_id"]] = a 
            print(e)

YEAR 2020
Total	TS-2020: 669
Processing...  Left:	 669
Processing...  Left:	 619
Processing...  Left:	 569
Processing...  Left:	 519
Processing...  Left:	 469
Processing...  Left:	 419
Processing...  Left:	 369
Processing...  Left:	 319
Processing...  Left:	 269
Processing...  Left:	 219
Processing...  Left:	 169
Processing...  Left:	 119
Processing...  Left:	 69
Processing...  Left:	 19
key:  7588
Dropping to db...


In [17]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user=USER,
  password=PASSWORD,
  database="mpp21"
)

mycursor = mydb.cursor()

In [18]:
def insert_article(a, curs, db):

    sql = "INSERT INTO articles (newspaper, article_id, title, content, summary, metadata, date, stage, endeavor, tone, tone_result, is_privacy) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (
        a["newspaper"], 
        a["article_id"],
        a["title"],
        a["content"],
        json.dumps(a["summary"]),
        json.dumps(a["metadata"]),
        a["date"],
        json.dumps(a["stage"]),
        json.dumps(a["endeavor"]),
        json.dumps(a["tone"]),
        json.dumps(a["tone_result"]),
        a["is_privacy"]
    )
    
    curs.execute(sql, val)

    db.commit()

    return

In [51]:
def select_content(curs, db):
    
    curs.execute("SELECT article_id FROM articles WHERE newspaper = 'TS'") 

    all_titles = []
    result = curs.fetchall()
    for r in result:
        all_titles.append(int(r[0][3:]))
    
    return all_titles

In [52]:
x = select_content(mycursor, mydb)

In [53]:
#x

In [54]:
max(x)

6918

In [ ]:
titls = []

for i in x.keys()

In [137]:
x

[]

In [45]:
# print("Dropping to db...")
# for a in articles.values():
#     try:
#         insert_article(a, mycursor, mydb)
#     except Exception as e: 
#         print(e)

## Estimate collection time

In [26]:
stats = pd.DataFrame.to_dict(current_stats)

In [30]:
stats["NZH"]

{2010: 0,
 2011: 0,
 2012: 0,
 2013: 0,
 2014: 0,
 2015: 0,
 2016: 0,
 2017: 0,
 2018: 0,
 2019: 0,
 2020: 0}

In [17]:
#stats = {}

# for n in newspapers.keys():
#     stats[n] = {}
#     for y in years:
#         stats[n][str(y)] = 0

In [22]:
#newspapers.keys()

In [36]:
# errors: NYT (blocked request)
# to do: REDO NYT
# try: The Dominion Post (Wellington, New Zealand)

n = "The Dominion Post (Wellington, New Zealand)"#"NYT"

#print_request = 1
#for y in [2010]: #years
fltr = "Source/Name eq '"+ n + "' and " + "year(Date) eq " + str(2010) + "" #newspapers[n]

# create request (top maxes at 50)
request_url = build_url(content='News', query=QUERY, skip=0, top=10, filter=fltr)

#if print_request:
print(request_url)
#    print_request = 0

r = requests.get(request_url, headers=request_headers)
print(r.content)
#     json_data = r.json() # extract data
#     total = get_result_count(json_data) # total articles per given year/newspaper

#   print("Total\t", n[:10], "-", y, ": ", total, sep="") 
#  stats[n][y] = total
#sleep(30)

https://services-api.lexisnexis.com/v1/News?$expand=Document&$search=privacy&$skip=0&$top=10&$filter=Source/Name eq 'The Dominion Post (Wellington, New Zealand)' and year(Date) eq 2010
b'{\r\n  "@odata.context":"https://services-api.lexisnexis.com/v1/$metadata#News","@odata.count":107,"value":[\r\n    {\r\n      "ResultId":"urn:contentItem:7Y5D-HTM1-2R6X-P014-00000-00","Jurisdiction":"U.S. Federal; International","Location":null,"ContentType":"News","Byline":null,"WordLength":472,"WebNewsUrl":null,"Geography":null,"NegativeNews":null,"Language":null,"Industry":null,"People":null,"Subject":null,"Section":"BUSINESS; Pg. 9","Company":null,"PublicationType":null,"Publisher":null,"GroupDuplicates":null,"InternationalLocation":null,"LEI":[\r\n        \r\n      ],"CompanyName":[\r\n        \r\n      ],"LNGI":[\r\n        \r\n      ],"SearchType":"NaturalLanguageAnd","Date":"2010-04-05T00:00:00Z","Keyword":null,"Title":"UN treaty on privacy possible","DocumentContent@odata.mediaReadLink":"Docu

In [72]:
with open('privacy_collection_stats.json', 'w') as fp:
    json.dump(stats, fp)

In [73]:
stats_df = pd.DataFrame.from_dict(stats)

In [74]:
stats_df

,AFR,AFR-O,DT,DT-O,GM,NYT,NZH,ODT,SMH,SMH-O,TG,TS,TS-O,USA
2010,240,0,580,67,506,511,545,0,557,0,1070,526,1,251
2011,206,0,562,537,392,377,551,0,659,0,1291,535,0,230
2012,223,0,738,1529,418,315,862,0,1010,0,1308,520,6,237
2013,182,0,691,1384,509,516,1102,0,646,244,1564,635,2,337
2014,283,0,575,1618,549,470,802,0,624,239,1384,1073,2,244
2015,207,10,586,1182,527,475,484,0,544,703,2236,718,1,218
2016,206,191,503,2022,573,471,562,0,530,588,2388,804,3,257
2017,254,161,448,937,466,230,681,0,371,362,739,519,180,190
2018,326,300,734,957,672,323,857,99,506,657,1195,812,566,239
2019,299,303,664,1200,543,377,967,73,436,536,1129,777,1511,188


In [75]:
stats_df.to_csv("privacy_stats.csv")

In [4]:
# L/N API helper functions provided by the database service 

def get_token(client_id, secret):
    """Gets Authorizaton token to use in other requests."""
    auth_url = 'https://auth-api.lexisnexis.com/oauth/v2/token' 
    payload = ('grant_type=client_credentials&scope=http%3a%2f%2f'
                'oauth.lexisnexis.com%2fall')
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    r = requests.post(
            auth_url,
            auth=HTTPBasicAuth(client_id, secret),
            headers=headers,
            data=payload)
    json_data = r.json()
    print("Token retrieved successfully.")
    return json_data['access_token']


def build_url(content='News', query='', skip=0, expand='Document', top=50, filter=None):
    """Builds the URL part of the request to Web Services API."""
    if filter != None:
        api_url = ('https://services-api.lexisnexis.com/v1/' + content +
                    '?$expand=' + expand + '&$search=' + query +
                    '&$skip=' + str(skip) + '&$top=' + str(top) +
                    '&$filter=' + filter)
    else:
        api_url = ('https://services-api.lexisnexis.com/v1/' + content +
                    '?$expand=' + expand + '&$search=' + query +
                    '&$skip=' + str(skip) + '&$top=' + str(top))
    return api_url



def build_header(token):
    """Builds the headers part of the request to Web Services API."""
    headers = {'Accept': 'application/json;odata.metadata=minimal',
                'Connection': 'Keep-Alive',
                'Host': 'services-api.lexisnexis.com'}
    headers['Authorization'] = 'Bearer ' + token
    return headers


def get_result_count(json_data):
    """Gets the number of results from @odata.count in the response"""
    return json_data['@odata.count']


def time_now():
    """Gets current time to the second."""
    now = datetime.now()
    return now.strftime('%Y-%m-%d-%H%M%S')